In [3]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image

# Load model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

def generate_caption(image_path):
    image = Image.open(image_path).convert("RGB")  # Open image
    inputs = processor(images=image, return_tensors="pt")  # Process image
    caption_ids = model.generate(**inputs)  # Generate caption
    return processor.decode(caption_ids[0], skip_special_tokens=True)  # Decode caption

# Example usage
image_path = "images.jpg"  # Replace with your image path
print("Caption:", generate_caption(image_path))


Caption: a pink flower in a garden


In [ ]:
import os
import torch
import asyncio
import nest_asyncio
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, CallbackContext

# Apply asyncio fix (for Jupyter Notebook or nested loops)
nest_asyncio.apply()

# Load the BLIP model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Telegram Bot Token (Replace with your actual token)
BOT_TOKEN = "7796800065:AAHFzZ_7lKNA2Ks1PmuG9_TEwJhB_sNc0IE"

# Start Command
async def start(update: Update, context: CallbackContext) -> None:
    await update.message.reply_text("Hello! Send me an image, and I'll generate a caption for you.")

# Handle Image Messages
async def handle_image(update: Update, context: CallbackContext) -> None:
    photo = update.message.photo[-1]  # Get the highest resolution photo
    file = await context.bot.get_file(photo.file_id)
    
    # Download the image
    image_path = "received_image.jpg"
    await file.download_to_drive(image_path)
    
    # Load and process the image
    raw_image = Image.open(image_path).convert("RGB")
    inputs = processor(raw_image, return_tensors="pt")
    
    # Generate caption
    with torch.no_grad():
        caption_ids = model.generate(**inputs)
    caption = processor.batch_decode(caption_ids, skip_special_tokens=True)[0]
    
    # Send the caption
    await update.message.reply_text(f"🖼 Caption: {caption}")

# Main function to set up the bot
def main():
    app = Application.builder().token(BOT_TOKEN).build()
    
    # Add handlers
    app.add_handler(CommandHandler("start", start))
    app.add_handler(MessageHandler(filters.PHOTO, handle_image))
    
    print("🤖 Bot is running...")
    app.run_polling()

# Run the bot
if __name__ == "__main__":
    main()


🤖 Bot is running...
